In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.interpolate as interpolate
import simulators.jla_supernovae.jla as jla
import simulators.jla_supernovae.jla_parser as jla_parser
import ndes.nde as nde
import distributions.priors as priors
%matplotlib inline

Using Theano backend.


In [3]:
### SET UP FOR SIMULATION CODE ###

# Import data
jla_data, jla_cmats = jla_parser.b14_parse(z_min=None, z_max=None, qual_cut=False,
                                    jla_path='simulators/jla_supernovae/jla_data/')
data = jla_data['mb']
delta_m_cut = 10
auxiliary_data = np.column_stack([jla_data['zcmb'], jla_data['x1'], jla_data['color'], np.array([(jla_data['3rdvar'] > delta_m_cut)], dtype=int)[0]])

# Om, w0, M_b, alpha, beta, delta_m
npar = 6
theta_fiducial = np.array([  0.20181324,  -0.74762939, -19.04253368,   0.12566322,   2.64387045, -0.05252869])

# Define prior limits and boundaries
lower = np.array([0, -1.5, -20, 0, 0, -0.5])
upper = np.array([0.6, 0, -18, 1, 6, 0.5])
Q = np.diag([0.4, 0.75, 0.1, 0.025, 0.25, 0.05])**2
Q[0,1] = Q[1,0] = -0.8*0.4*0.75
Qinv = np.linalg.inv(Q)
prior_mean = np.array([  0.3  ,  -0.75 , -19.05 ,   0.125,   2.6  ,  -0.05 ])
prior_args = [prior_mean, Q, lower, upper]

# Covariance matrix
C = jla_parser.b14_covariance(jla_data, jla_cmats, theta_fiducial[3], theta_fiducial[4])
Cinv = np.linalg.inv(C)
L = np.linalg.cholesky(C)

# Derivative of the covariance matrix
n_sn = len(C)
dCdt = np.zeros((npar, n_sn, n_sn))

# Step size for derivatives
step = abs(0.01*theta_fiducial)

# N data points
ndata = len(jla_data['mb'])

# Simulation args
sim_args = [auxiliary_data, L]

# Compute the mean
mu = jla.apparent_magnitude(theta_fiducial, auxiliary_data)

# Compute the derivatives
dmdt = jla.dmudtheta(theta_fiducial, jla.simulation_seeded, step, npar, ndata, sim_args)
dmdt[2,:] = np.ones(n_sn)
dmdt[3,:] = -jla_data['x1']
dmdt[4,:] = jla_data['color']
dmdt[5,:] = (jla_data['3rdvar'] > 10)

# Fisher matrix
F, Finv = jla.fisher(dmdt, dCdt, Cinv, Qinv, npar)
fisher_errors = np.sqrt(np.diag(Finv))

# Compute projection vectors
Fpinv = np.linalg.inv(F[2:,2:])
P1 = np.dot(Fpinv, F[0,2:])
P2 = np.dot(Fpinv, F[1,2:])

# Simulation args for ABC
compressor_args = [theta_fiducial, Finv, Cinv, dmdt, dCdt, mu, Qinv, prior_mean, F, P1, P2]

# Parameter names for plotting
names = ['\Omega_m', 'w_0', 'M_\mathrm{B}', '\alpha', '\beta', '\delta M']
labels =  ['\\Omega_m', 'w_0', 'M_\mathrm{B}', '\\alpha', '\\beta', '\\delta M']
ranges = {'\Omega_m':[lower[0], upper[0]], '\w0':[lower[1], upper[1]]}

# Compressed dataset
data = jla.compressor_projected(data, compressor_args)

# Define new separate priors over eta and theta
theta_fiducial = np.array([0.20181324,  -0.74762939])
Finv = Finv[0:2,0:2]
lower = np.array([0, -1.5])
upper = np.array([0.6, 0])
Q = np.diag([0.4, 0.75])**2
Q[0,1] = Q[1,0] = -0.8*0.4*0.75
Qinv = np.linalg.inv(Q)
prior_mean = np.array([  0.3  ,  -0.75])
prior = priors.TruncatedGaussian(prior_mean, Q, lower, upper)

eta_prior = priors.TruncatedGaussian(np.array([-19.05, 0.125, 2.6, -0.05]), 
                                     np.diag([0.1, 0.025, 0.25, 0.05])**2, 
                                     np.array([-20, 0, 0, -0.5]),
                                     np.array([-18, 1, 6, 0.5]))


/Users/jalsing/Dropbox (Simons Foundation)/science/delfi/DELFI_MASTER/delfi/simulators/jla_supernovae/jla_parser.py:9: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  dtype = None, names = True)


In [4]:
# Define the simulator function: takes parameters, spits out simulated data
# Should have the form: simulator(parameters, args) -> simulated dataset
def simulator(theta, simulator_args):
    
    eta = eta_prior.draw()
    return jla.simulation(np.concatenate([theta, eta]), simulator_args)
simulator_args = sim_args

In [5]:
# Define the compression function: takes data, spits out compressed summaries
# Should have the form compressor(data, args) -> compressed summaries
# NB: compression should be set-up like a quasi maximum-likelihood estimator
compressor = jla.compressor_projected
compressor_args = compressor_args

In [ ]:
# Create the DELFI MDN object
n_components = 3

mdn = nde.DelfiMixtureDensityNetwork(data, prior, [lower, upper], Finv, theta_fiducial, n_components, n_hidden = [50, 50], activations = ['tanh', 'tanh'], names = names, labels = labels, ranges = ranges, results_dir='simulators/jla_supernovae/results_marginalized/')

In [ ]:
# Do the Fisher pre-training
mdn.fisher_pretraining(50000, prior, epochs=100)

Generating fisher pre-training data...
Training on the pre-training data...
Train on 45000 samples, validate on 5000 samples
Epoch 1/100
44900/45000 [============================>.] - ETA: 0s - loss: 1.9296

In [ ]:
# Proposal for the SNL
proposal = priors.TruncatedGaussian(theta_fiducial, 9*Finv, lower, upper)

# Initial samples, batch size for population samples, number of populations
n_initial = 100
n_batch = 100
n_populations = 99

# Do the SNL training
mdn.sequential_training(simulator, compressor, n_initial, n_batch, n_populations, proposal, simulator_args=simulator_args, compressor_args=compressor_args)

In [ ]:
# Trace plot of the loss as a function of the number of simulations
plt.scatter(mdn.n_sim_trace, mdn.loss_trace, s = 20)
plt.plot(mdn.n_sim_trace, mdn.loss_trace, color = 'red')
plt.xlim(0, mdn.n_sim_trace[-1])
plt.xlabel('number of simulations')
plt.ylabel('loss')
plt.show()